# Extraction of landscape geometry features

In [ ]:
import copy
import os

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import pyvista as pv
from scipy import spatial
from scipy import stats
import seaborn as sns

In [ ]:
sns.set()

## Load the original data

Obtained from F. Poux's [video tutorial](https://www.youtube.com/watch?v=WKSJcG97gE4) on 3D point cloud feature extraction, available on [Google Drive](https://drive.google.com/drive/folders/1fwhE5OphpeW4RR0RY8W2jbqmlf5LH6dX).

In [ ]:
pcd_pv = pv.read(os.path.join('data', 'MLS_UTWENTE_super_sample.ply'))
pcd_pv['elevation'] = pcd_pv.points[:, 2]
pv.plot(pcd_pv,
        scalars='elevation',
        render_points_as_spheres=True,
        point_size=2,
        show_scalar_bar=False)

## Create the point cloud

In [ ]:
points = np.asarray(pcd_pv.points)
points[:, 2] -= points[:, 2].min()  # set min elevation to 0
norm = plt.Normalize()
elev = plt.cm.viridis(norm(points[:, 2]))[:, :-1]  # add color
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(points)
pcd_o3d.colors = o3d.utility.Vector3dVector(elev)

In [ ]:
o3d.visualization.draw_geometries([pcd_o3d])

## Exploring the unstructured data with an octree

In [ ]:
curr_max_depth = 7
octree = o3d.geometry.Octree(max_depth=curr_max_depth)
octree.convert_from_point_cloud(pcd_o3d, size_expand=0.01)

In [ ]:
o3d.visualization.draw_geometries([octree])

## Downsampling the point cloud using a voxel grid

In [ ]:
pcd_ds = pcd_o3d.voxel_down_sample(voxel_size=0.8)  # cca. 10x downsample

In [ ]:
o3d.visualization.draw_geometries([pcd_ds])

## Performing outlier removal

In [ ]:
def display_inlier_outlier(cloud, ind):
    if not isinstance(cloud, o3d.geometry.PointCloud):
        points = cloud
        cloud = o3d.geometry.PointCloud()
        cloud.points = o3d.utility.Vector3dVector(np.asarray(points))
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
pcd_stat, ind = pcd_ds.remove_statistical_outlier(nb_neighbors=30,
                                                  std_ratio=3)

In [ ]:
display_inlier_outlier(pcd_ds, ind)

In [ ]:
pcd_rad, ind = pcd_ds.remove_radius_outlier(nb_points=25, radius=5)

In [ ]:
display_inlier_outlier(pcd_ds, ind)

In [ ]:
o3d.visualization.draw_geometries([pcd_rad])

## Extracting geometric features

In [ ]:
X = np.asarray(pcd_rad.points)

### 3D

In [ ]:
def pca(X):
    X_norm = X - np.mean(X, axis=0)
    cov = np.cov(X_norm, rowvar=False)
    eval, evec = np.linalg.eig(cov)
    mask = np.argsort(eval)[::-1]
    return eval[mask], evec[mask]

In [ ]:
def extract_features(eval, evec):
    # https://doi.org/10.5194/isprsannals-II-5-W2-313-2013
    planarity = (eval[1] - eval[2]) / eval[0]
    linearity = (eval[0] - eval[1]) / eval[0]
    omnivariance = (eval[0] * eval[1] * eval[2]) ** (1 / 3)
    _, _, normal = evec
    verticality = 1 - normal[2]
    return (planarity, linearity, omnivariance, verticality,
            normal[0], normal[1], normal[2])

In [ ]:
tree = spatial.KDTree(X)
dist, ind = tree.query(X, k=25)
nbhd = X[ind]

In [ ]:
# example for a single point
sel = 0
eval, evec = pca(nbhd[sel])
p, l, o, v, nx, ny, nz = extract_features(eval, evec)

In [ ]:
p, l, o, v, nx, ny, nz

In [ ]:
# surface normals
n = np.empty_like(X)
for i in range(X.shape[0]):
    eval, evec = pca(nbhd[i, ...])
    _, _, _, _, nx, ny, nz = extract_features(eval, evec)
    n[i, :] = [nx, ny, nz]
pcd_rad.normals = o3d.utility.Vector3dVector(n)
pcd_rad.orient_normals_consistent_tangent_plane(20)

In [ ]:
o3d.visualization.draw_geometries([pcd_rad], point_show_normal=True)

### 2D

In [ ]:
def display_selection(cloud, ind):
    if not isinstance(cloud, o3d.geometry.PointCloud):
        points = cloud
        cloud = o3d.geometry.PointCloud()
        cloud.points = o3d.utility.Vector3dVector(np.asarray(points))
    selected = cloud.select_by_index(ind)
    unselected = cloud.select_by_index(ind, invert=True)
    selected.paint_uniform_color([1, 0, 0])
    unselected.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([selected, unselected])

In [ ]:
tree_2d = spatial.KDTree(X[:, :2])
ind_2d = tree_2d.query_ball_point(X[:, :2], 4)

In [ ]:
# example for a single selection
sel = 0
X_sel = X[ind_2d[sel]]

In [ ]:
display_selection(X, ind_2d[sel])

In [ ]:
# create a distribution of elevations of the scenary
elevs = []
for i in range(X.shape[0]):
    X_sel = X[ind_2d[i]]
    elevs.append(X_sel[:, 2].ptp())

In [ ]:
kernel = stats.gaussian_kde(elevs)

In [ ]:
fig, ax = plt.subplots()
y, bins, patches = ax.hist(elevs,
                           bins='fd',
                           density=True,
                           cumulative=False,
                           histtype='bar',
                           align='mid',
                           orientation='vertical',
                           label='measured data')
ax.plot(bins, kernel(bins), label='kernel density estimate')
ax.set(xlabel='elevation (m)', ylabel='probability density')
ax.legend();

In [ ]:
kernel.integrate_box_1d(min(elevs), max(elevs))

In [ ]:
kernel.covariance

In [ ]:
kernel.covariance_factor()

## Performing a simple semantic segmentation of the flat terrain

In [ ]:
models = {}
segments = {}
n_planes = 1

In [ ]:
pcd_rest = copy.deepcopy(pcd_rad)
pcd_rest.paint_uniform_color([0.8, 0.8, 0.8])
colors = sns.color_palette(n_colors=n_planes+1)

In [ ]:
for i in range(n_planes):
    models[i], ind = pcd_rest.segment_plane(distance_threshold=0.5,
                                            ransac_n=3,
                                            num_iterations=1000)
    segments[i] = pcd_rest.select_by_index(ind)
    segments[i].paint_uniform_color(colors[i])
    pcd_rest = pcd_rest.select_by_index(ind, invert=True)
pcd_rest.paint_uniform_color(colors[-1]);

In [ ]:
o3d.visualization.draw_geometries(list(segments.values()) + [pcd_rest])

In [ ]:
a, b, c, d = models[0]
print(f'implicit eqn. {a:.2e} x + {b:.2e} y + {c:.2e} z + {d:.2e} = 0')

In [ ]:
obb = segments[0].get_oriented_bounding_box()
obb.color = colors[-1]

In [ ]:
# after manaully cleaning the plane point cloud, it is easy to estimate
# its approximate surface area e.g. in Meshlab
o3d.visualization.draw_geometries(list(segments.values()) + [obb])